In [8]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
%matplotlib inline


In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [10]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

OSError: SavedModel file does not exist at: C:\Users\akash\AppData\Local\Temp\tfhub_modules\312f001449331ee3d410d758fccdc9945a65dbc3\{saved_model.pbtxt|saved_model.pb}

In [ ]:
img=cv.imread('cr7.jpg')
cv.imshow('demo',img)
cv.waitKey(0)

In [ ]:
input_img=img.copy()
input_img=cv.resize(input_img,(608,384))
input_img=np.array(input_img,dtype='int32')
input_img=tf.convert_to_tensor(input_img)
input_img=tf.expand_dims(input_img,axis=0)


In [10]:
outputs=movenet(input_img)

In [11]:
outputs=outputs['output_0'].numpy()


In [12]:
def get_keypoint_dict(outputs):
    outputs=outputs['output_0'].numpy()
    keypoints_dict={'Person_1':{},'Person_2':{},'Person_3':{},'Person_4':{},'Person_5':{},'Person_6':{}}
    for num,key in enumerate(keypoints_dict.keys()):
        keypoints_dict[key]['keypoints']=outputs[:,num,:51].reshape(17,3)
        keypoints_dict[key]['bbox']=outputs[:,num,51:].flatten()
    return keypoints_dict

    
    

In [13]:
def draw_keypoints(img,keypoints_dict,confidence):
    Y,X,c=img.shape
    for num,key in enumerate(keypoints_dict.keys()):
        for y,x,conf in keypoints_dict[key]['keypoints']:
            if conf>confidence:
                cv.circle(img,(int(X*x),int(Y*y)),6, (0,255,0), -1)
            
        

In [14]:
draw_keypoints(img,keypoints_dict,0.2)
cv.imshow('demo',img)
cv.waitKey(0)

NameError: name 'keypoints_dict' is not defined

In [15]:
cap=cv.VideoCapture('dance.mp4')
while cap.isOpened():
    success,img=cap.read()
    if success:
        input_img=img.copy()
        input_img=cv.resize(input_img,(608,320))
        input_img=np.array(input_img,dtype='int32')
        input_img=tf.convert_to_tensor(input_img)
        input_img=tf.expand_dims(input_img,axis=0)
        outputs=movenet(input_img)
        keypoints_dict=get_keypoint_dict(outputs)
        draw_keypoints(img,keypoints_dict,0.2)
        cv.imshow('demo',img)
        if cv.waitKey(1) & 0xFF==ord('q'):
            break
            
    else:
        break
cap.release()
cv.destroyAllWindows()
        

In [18]:
keypoints_dict['Person_1']['bbox']

array([0.19531295, 0.3940979 , 0.9924828 , 0.75689316, 0.16702023],
      dtype=float32)

In [15]:
1080/1920

0.5625

In [ ]:
a